## 11. Network and Web Programming

### 11.1 HTTP Service

搞清楚几个概念：

1. 网络连接。 套接字（socket），七层协议。

TCP/IP层。 IP 网络层，负责将包发送到指定的地方；  TCP： 传输控制层。

python 用作url请求的库有：

- urllib
- requests


### 11.2 TCP Server

python自带 ：  `socketserver.TCPServer`

如果用一些web框架，比如有 `flask`， `Django` 等等

In [2]:
from socketserver import BaseRequestHandler, TCPServer

class EchoHandler(BaseRequestHandler):
    def handle(self):
        print('Got connection from', self.client_address)
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)

def main():
    serv = TCPServer(('', 20000), EchoHandler)
    print('Echo server running on port 20000')
    serv.serve_forever()


下面的例子用了若干线程，起了若干server。

In [4]:
from socketserver import StreamRequestHandler, TCPServer

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print('Got connection from', self.client_address)
        # self.rfile is a file-like object for reading
        for line in self.rfile:
            # self.wfile is a file-like object for writing
            self.wfile.write(line)

def main():
    from threading import Thread
    NWORKERS = 16
    serv = TCPServer(('', 20000), EchoHandler)
    for n in range(NWORKERS):
        t = Thread(target=serv.serve_forever)
        t.daemon = True
        t.start()
    print('Multithreaded server running on port 20000')
    serv.serve_forever()


### 11.3 UDP Server



In [5]:
from socketserver import BaseRequestHandler, UDPServer
import time

class TimeHandler(BaseRequestHandler):
    def handle(self):
        print('Got connection from', self.client_address)
        # Get message and client socket
        msg, sock = self.request
        resp = time.ctime()
        sock.sendto(resp.encode('ascii'), self.client_address)

def main():
    serv = UDPServer(('', 20000), TimeHandler)
    serv.serve_forever()



### 11.4 IP地址的一些操作

python库 有 `ipaddress`

In [8]:
import ipaddress

net = ipaddress.ip_network('123.45.67.64/27')

for a in net:
    print(a)

123.45.67.64
123.45.67.65
123.45.67.66
123.45.67.67
123.45.67.68
123.45.67.69
123.45.67.70
123.45.67.71
123.45.67.72
123.45.67.73
123.45.67.74
123.45.67.75
123.45.67.76
123.45.67.77
123.45.67.78
123.45.67.79
123.45.67.80
123.45.67.81
123.45.67.82
123.45.67.83
123.45.67.84
123.45.67.85
123.45.67.86
123.45.67.87
123.45.67.88
123.45.67.89
123.45.67.90
123.45.67.91
123.45.67.92
123.45.67.93
123.45.67.94
123.45.67.95


### 11.5 REST API

关于web编程也有一些概念，比如 WSGI。

一些问题：

1. 什么是 web server？
2. apache 和nginx的区别？
3. 什么是WSGI？
4. WSGI框架



In [11]:
#web_application.py
from wsgiref.simple_server import make_server

def application(environ, start_response):
    path = environ.get('PATH_INFO')
    if path == '/':
        response_body = "Index"
    else:
        response_body = "Hello"
    status = "200 OK"
    response_headers = [("Content-Length", str(len(response_body)))]
    start_response(status, response_headers)
    return [response_body]

def main():
    httpd = make_server(
        '127.0.0.1', 8051, application)

    httpd.serve_forever()
    

### 11.6 Simple Remote Procedure Call

XML-RPC

这个概念没有遇到过。略。

### 11.7 Communicating Simply Between Interpreters 解释器间通信

也没有遇到过。 略。

### 11.8 RemoteProcedure Call， RPC


### 11.10 Adding SSL to Network Services

### 11.11 进程间传递 Socket File Descriptor


### 11.12 Understanding Event-Driven I/O

事件驱动IO。 轮询机制。

计算机科学解决的大部分问题是 如何最大化利用硬件资源的问题。 这里面又有一大块是并发的问题。 

如果能用，就绝不让他闲着。就是这样的原则。。。

对于一些 I/O操作， 通常感觉是很慢，容易阻塞，比如网络请求。等待的时候，可以做些其他事情。 但什么时候再回来。 这样会有一些机制。

事件驱动型I/O，的原理实际上是一种 轮询机制： 任务派下去之后， 询问各代表，事情有没有做完？ 如果做完了，继续下一步


下面的例子只是描述了这样的一个概念。

关于select、poll、epoll见 <https://segmentfault.com/a/1190000003063859> 写的很好。

In [12]:
class EventHandler:
    def fileno(self):
        'Return the associated file descriptor'
        raise NotImplemented('must implement')

    def wants_to_receive(self):
        'Return True if receiving is allowed'
        return False

    def handle_receive(self):
        'Perform the receive operation'
        pass

    def wants_to_send(self):
        'Return True if sending is requested' 
        return False

    def handle_send(self):
        'Send outgoing data'
        pass

import select

def event_loop(handlers):
    while True:
        wants_recv = [h for h in handlers if h.wants_to_receive()]
        wants_send = [h for h in handlers if h.wants_to_send()]
        can_recv, can_send, _ = select.select(wants_recv, wants_send, [])
        for h in can_recv:
            h.handle_receive()
        for h in can_send:
            h.handle_send()



udp server是一种无连接的server， 不会保持连接。

tcp server是一种保持连接的server。除非client主动断掉或者其他异常。


### 11.13 Sending and Receiving Large Arrays

发送和接受大数组。

可能是因为 内存的限制，无法一次copy太多。

同时为了最大化减少copy的次数。
